In [22]:
import requests
import os
import re
import json
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from selenium import webdriver
import time
import matplotlib.pyplot as plt
import urllib
%matplotlib inline
from datetime import date

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
import time


In [23]:

# Setup the ChromeDriver with a valid User-Agent
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Optional, to run in headless mode
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')


# Set a valid User-Agent header
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
options.add_argument(f'user-agent={user_agent}')

driver = webdriver.Chrome(options=options)

headers = {
    'Accept-Language': 'en',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'X-FORWARDED-FOR': '2.21.184.0'
}

In [24]:
pmg = 'https://pmg.org.za/question_replies/'

In [25]:
response = requests.get(pmg, headers=headers)
print(response.status_code)

200


In [26]:
# pmg_soup = BeautifulSoup(response.text, 'html.parser')

In [27]:
# def safe_extract(soup, selector, attribute, default=np.nan):
#         """Helper function to extract text from a BeautifulSoup tag safely."""
#         element = soup.find(selector, attribute)
#         return element.get_text(strip=True) if element else default


# # Helper function to extract text from multiple <p> tags within a specific div
# def safe_extract_multiple(soup, parent_selector, parent_attribute, child_selector='p'):
#     """Helper function to extract text from multiple <p> tags safely inside a parent container."""
#     parent_element = soup.find(parent_selector, parent_attribute)
#     if parent_element:
#         child_elements = parent_element.find_all(child_selector)
#         return [element.get_text(strip=True) for element in child_elements]
#     return []



In [28]:
def fetch_soup(pmg_url, headers):
    """
    Fetch the content of a URL (base URL + movie link) and return a BeautifulSoup object.

    Args:
        base_url (str): The base URL (e.g., 'https://www.imdb.com/').
        movie_link (str): The relative URL or path to the movie page.
        headers (dict): Headers to be used in the HTTP request.

    Returns:
        BeautifulSoup: Parsed HTML content of the page if successful, else None.
    """
    

    # Fetch the page content
    response = requests.get(pmg_url, headers=headers)
    
    # Check for a successful response and return BeautifulSoup object
    if response.status_code == 200:
        return BeautifulSoup(response.text, 'html.parser')
    
    return None


In [29]:
pmg_soup = fetch_soup('https://pmg.org.za/question_replies/9', headers)

In [30]:
# Helper function to extract text from a single tag
def safe_extract(soup, selector, attribute, default=np.nan):
    """Helper function to extract text from a BeautifulSoup tag safely."""
    element = soup.find(selector, attribute)
    return element.get_text(strip=True) if element else default


# Helper function to extract text from multiple <p> tags within a specific div
def safe_extract_multiple(soup, parent_selector, parent_attribute, child_selector='p'):
    """Helper function to extract text from multiple <p> tags safely inside a parent container."""
    parent_element = soup.find(parent_selector, parent_attribute)
    if parent_element:
        child_elements = parent_element.find_all(child_selector)
        return [element.get_text(strip=True) for element in child_elements]
    return []

In [31]:
# Extract the required fields
pmg_date = safe_extract(pmg_soup, 'h4', {}, default=np.nan)  # Extracts the date and NW552 reference
pmg_credentials = safe_extract(pmg_soup, 'h4', {'class': 'media-heading'}, default=np.nan)  # Extract the name and credentials of the person asking
# pmg_credentials = safe_extract(pmg_soup, 'h4', {'class': 'media-body'}, default=np.nan)  # Extract the name and credentials of the person asking
pm_question = safe_extract(pmg_soup, 'p', {}, default=np.nan)  # Extract the question text

# Extract the content from all <p> tags within the specific 'committee-question-reply' div
pmg_answer_paragraphs = safe_extract_multiple(pmg_soup, 'div', {'class': 'committee-question-reply'})

# Join paragraphs into one string for better readability
pmg_answer = " ".join(pmg_answer_paragraphs)



# Output the extracted values
print("Date:", pmg_date)
print("\nCredentials:", pmg_credentials)
print("\nQuestion:", pm_question)
print("\nAnswer:", pmg_answer)

Date: 18 September 2024 -CW103

Credentials: Chirwa-Mpungose, Ms NNto ask the
Minister of Social Development:

Question: (a) What new interventions has her department initiated and planned as the government has failed to address gender-based violence and femicide (details furnished), (b) what has been the fundamental studies to justify such programmes and (c) what impact could research indicate that they are ensuring (i) prevention, (ii) education and (iii) contribution to end violence against women and children?

Answer: Find replyhere


In [32]:
def get_pmg_content(pmg_soup):


    # Extract the required fields from 'committee-question-detail'
    pmg_date = safe_extract(pmg_soup, 'h4', {}, default=np.nan)  # Extract the date and NW552 reference from the <h4> tag
    
    # pmg_credentials = safe_extract(pmg_soup, 'div', {'class': 'media-body'}, default=np.nan)  # Extract name and credentials of the person asking from 'media-body' class
    pmg_credentials = safe_extract(pmg_soup, 'h4', {'class': 'media-heading'}, default=np.nan)

    pm_question = safe_extract(pmg_soup, 'p', {}, default=np.nan)  # Extract the question from the first <p> tag in the 'committee-question-detail' div
    # Extract the content from all <p> tags within the 'committee-question-reply' div
    pmg_answer_paragraphs = safe_extract_multiple(pmg_soup, 'div', {'class': 'committee-question-reply'})

    # Join paragraphs into one string for better readability
    pmg_answer = " ".join(pmg_answer_paragraphs)

    return pmg_date, pmg_credentials, pm_question, pmg_answer

In [33]:

def get_all_pmg_content(pages=5, headers=headers, verbose=False):
    columns = ['date', 'mp', 'question', 'answer']
    data = []
    done = 0
    base_url = 'https://pmg.org.za/question_replies/'

    start = time.time()
    
    for page in range(1, pages):
        
        soup = fetch_soup(f'{base_url}{page}', headers)
        
        if soup:
            pages_stat = list(get_pmg_content(soup))
            # data.append([page] + pages_stat)
            data.append(pages_stat)
        else:
            # data.append([page] + [np.nan] * 4)
            data.append([np.nan] * 4)
        
        if verbose:
            done += 1
            print(f'Done with {done} pages. {pages - done} left. Elapsed time: {(time.time() - start):.1f} seconds')
        
        time.sleep(1)  # To avoid overwhelming the server
    
    return pd.DataFrame(data, columns=columns)
                  


In [34]:

def get_all_pmg_content(pages=5, headers=None, verbose=False):
    columns = ['date', 'mp', 'question', 'answer']
    data = []
    done = 0
    base_url = 'https://pmg.org.za/question_replies/'

    start = time.time()

    # Step 1: Scrape the base page first (without page number)
    soup = fetch_soup(base_url, headers)
    if soup:
        # Find all 'committee-question-detail' elements on the base page
        question_details = soup.select('article > section .committee-question-detail')
        
        # For each 'committee-question-detail' element, scrape the content
        for detail in question_details:
            content = get_pmg_content(detail)  # Extract the content for each detail
            data.append(content)  # Append the extracted content to the data list
    else:
        # If no content is found on the base page, append NaNs
        data.append([np.nan] * 4)

    # Step 2: Loop through additional pages (1, 2, ..., n)
    for page in range(1, pages + 1):  # Loop through specified numbered pages
        soup = fetch_soup(f'{base_url}{page}', headers)  # Fetch soup for each numbered page

        if soup:
            # Find all 'committee-question-detail' elements on the numbered pages
            question_details = soup.select('article > section .committee-question-detail')
            
            # For each 'committee-question-detail' element, scrape the content
            for detail in question_details:
                content = get_pmg_content(detail)  # Extract the content for each detail
                data.append(content)  # Append the extracted content to the data list
        else:
            # If no content is found, append NaNs to maintain consistent rows
            data.append([np.nan] * 4)

        # Print verbose progress
        if verbose:
            done += 1
            print(f'Done with {done} pages. {pages - done} left. Elapsed time: {(time.time() - start):.1f} seconds')

        time.sleep(1)  # Pause to avoid overwhelming the server

    # Return the scraped data as a DataFrame
    return pd.DataFrame(data, columns=columns)


In [35]:
_sample_raw = get_all_pmg_content(pages=5, headers=headers, verbose=1)

Done with 1 pages. 4 left. Elapsed time: 3.2 seconds
Done with 2 pages. 3 left. Elapsed time: 6.1 seconds
Done with 3 pages. 2 left. Elapsed time: 9.3 seconds
Done with 4 pages. 1 left. Elapsed time: 11.8 seconds
Done with 5 pages. 0 left. Elapsed time: 14.4 seconds


In [36]:

_sample_clean = _sample_raw.copy()
_sample_clean.head()

,date,mp,question,answer
0,11 October 2024 -NW801,"Bodlani, Ms Tto ask the\nMinister of Communica...",Whether his department has put any plans in pl...,Find replyhere
1,11 October 2024 -NW223,"Moela, Mr MSto ask the\nMinister of Police",How will the SA Police Service develop mechani...,The workload of the Detectives is created by a...
2,11 October 2024 -NW990,"Ndlozi, Dr MQto ask the\nMinister of Finance","What total (a) amount in State incentives, suc...",The automobile industry benefits from the Auto...
3,11 October 2024 -NW771,"Mrara, Mr Mto ask the\nMinister of Communicati...",(a) What is the long-term strategy of his depa...,I have been advised by the Department as follo...
4,11 October 2024 -NW909,"Mdluli, Mr MMto ask the\nMinister of Communica...",Since the start of the migration from analogue...,I have been advised by the Department as follo...


In [37]:
# Step 1: Extract the ID from the original 'date' column (before converting to datetime)
_sample_clean['id'] = _sample_clean['date'].str.extract(r'-([A-Z0-9]+)')

# Step 2: Clean up the `date` column to remove anything after the date
_sample_clean['date'] = _sample_clean['date'].str.extract(r'^([0-9]{2} [A-Za-z]+ [0-9]{4})')

# Step 3: Convert the cleaned date column to datetime format
_sample_clean['date'] = pd.to_datetime(_sample_clean['date'], format='%d %B %Y')

# Step 4: Split the 'mp' column using '\n' and get the first part as 'mp' and last part as 'responder'
_sample_clean['mp'], _sample_clean['responder'] = _sample_clean['mp'].str.split('\n').str[0], _sample_clean['mp'].str.split('\n').str[1]

# Step 5: Clean up the 'mp' column by removing 'to ask the'
_sample_clean['mp'] = _sample_clean['mp'].str.replace('to ask the', '', regex=False)



          date                   mp  \
0   2024-10-11        Bodlani, Ms T   
1   2024-10-11         Moela, Mr MS   
2   2024-10-11        Ndlozi, Dr MQ   
3   2024-10-11          Mrara, Mr M   
4   2024-10-11        Mdluli, Mr MM   
..         ...                  ...   
295 2024-09-27         Blose, Mr MB   
296 2024-09-27   Mente-Nkuna, Ms NV   
297 2024-09-27         Mathys, Ms L   
298 2024-09-27  Schickerling, Ms LM   
299 2024-09-27         Mogale, Mr T   

                                              question  \
0    Whether his department has put any plans in pl...   
1    How will the SA Police Service develop mechani...   
2    What total (a) amount in State incentives, suc...   
3    (a) What is the long-term strategy of his depa...   
4    Since the start of the migration from analogue...   
..                                                 ...   
295  (a) What (i) is the reason that a tender of R9...   
296  What are the reasons that (a) no statutory rap...   
297  What

In [38]:
_sample_clean['mp'][1]

'Moela, Mr MS'

In [39]:
# Display the cleaned DataFrame
_sample_clean.head()


,date,mp,question,answer,id,responder
0,2024-10-11,"Bodlani, Ms T",Whether his department has put any plans in pl...,Find replyhere,NW801,Minister of Communications and Digital Technol...
1,2024-10-11,"Moela, Mr MS",How will the SA Police Service develop mechani...,The workload of the Detectives is created by a...,NW223,Minister of Police
2,2024-10-11,"Ndlozi, Dr MQ","What total (a) amount in State incentives, suc...",The automobile industry benefits from the Auto...,NW990,Minister of Finance
3,2024-10-11,"Mrara, Mr M",(a) What is the long-term strategy of his depa...,I have been advised by the Department as follo...,NW771,Minister of Communications and Digital Technol...
4,2024-10-11,"Mdluli, Mr MM",Since the start of the migration from analogue...,I have been advised by the Department as follo...,NW909,Minister of Communications and Digital Technol...


In [40]:
_sample_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   date       300 non-null    datetime64[ns]
 1   mp         300 non-null    object        
 2   question   300 non-null    object        
 3   answer     300 non-null    object        
 4   id         300 non-null    object        
 5   responder  300 non-null    object        
dtypes: datetime64[ns](1), object(5)
memory usage: 14.2+ KB


In [41]:
# Convert the dictionary into a DataFrame
# _2024_films_links_df = pd.DataFrame(list(_2024_films_links.items()), columns=['Title', 'Link'])

# Write the DataFrame to a Parquet file with Brotli compression
_sample_clean.to_parquet('_sample_clean.parquet.brotli', compression='brotli')

print("Data written to parquet successfully.")


Data written to parquet successfully.
